In [18]:
import sys
import pandas as pd
import numpy as np
import xarray as xr
sys.path.insert(0, '../..')
from config import DATA_DIR
unique_years = np.arange(2003, 2022)

yearly_data = []

# Load and preprocess the data for each year
for year in unique_years:
    file = DATA_DIR / f"hazard_input_data/isimip3a/flood_fraction/cama-flood_ensemble_median_20crv3-era5_obsclim_histsoc_default_fldfrc_15arcmin_global_month-max_nopros_{year}.nc"
    yearly_hazard = xr.open_dataset(file)
    yearly_hazard = yearly_hazard.sel(time=str(year))
    yearly_hazard = yearly_hazard.rename({"lat": "latitude", "lon": "longitude"})
    
    # Select the flood data variable
    flood_data = yearly_hazard['fldfrc']  
    
    # Append the data to the list
    yearly_data.append(flood_data)

# Concatenate all yearly data along a new time dimension
combined_data = xr.concat(yearly_data, dim="time")

# Calculate the median across the time dimension (years)
median_flood = combined_data.median(dim="time")

In [16]:
yearly_hazard

<xarray.Dataset>
Dimensions:    (time: 12, bnds: 2, longitude: 1440, latitude: 720)
Coordinates:
  * time       (time) datetime64[ns] 2005-01-16T12:00:00 ... 2005-12-16
  * longitude  (longitude) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * latitude   (latitude) float32 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    fldfrc     (time, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    CDI:          Climate Data Interface version 1.9.5 (http://mpimet.mpg.de/...
    history:      Thu May 23 21:25:10 2024: cdo -L -O enspctl,50 /p/tmp/sazim...
    Conventions:  CF-1.6
    frequency:    mon
    CDO:          Climate Data Operators version 1.9.5 (http://mpimet.mpg.de/...

In [19]:
# Iterate over each year's data to set values below the median to 0
for i, year in enumerate(unique_years):
    yearly_data[i] = yearly_data[i].where(yearly_data[i] >= median_flood, 0)

    # Save the modified yearly data to a new NetCDF file
    output_file = DATA_DIR / f"hazard_input_data/isimip3a/flood_fraction/cama-flood_ensemble_median_20crv3-era5_obsclim_histsoc_default_fldfrc_15arcmin_global_month-max_2y_{year}.nc"
    yearly_data[i].to_netcdf(output_file)
